In [1]:
import pandas as pd 
df=pd.read_csv('lab4_data.csv')
df

,age,income,student,credit_rating,buys_computer
0,<=30,high,no,fair,no
1,<=30,high,no,excellent,no
2,31…40,high,no,fair,yes
3,>40,medium,no,fair,yes
4,>40,low,yes,fair,yes
5,>40,low,yes,excellent,no
6,31…40,low,yes,excellent,yes
7,<=30,medium,no,fair,no
8,<=30,low,yes,fair,yes
9,>40,medium,yes,fair,yes


In [2]:
#A1 claculating prior probability for each class
count_classes=df['buys_computer'].value_counts()
prob_yes=count_classes['yes']/len(df)
prob_no=count_classes['no']/len(df)
print("probability of yes for buys_computer is ",prob_yes)
print("probability of no for buys_computer is ", prob_no)

probability of yes for buys_computer is  0.6428571428571429
probability of no for buys_computer is  0.35714285714285715


In [3]:
df.columns 

Index(['age ', 'income', 'student', 'credit_rating', 'buys_computer'], dtype='object')

In [4]:
df.dtypes 


age              object
income           object
student          object
credit_rating    object
buys_computer    object
dtype: object

In [8]:
from scipy.stats import norm
def clean_numeric_age(value):
    if isinstance(value, str):
        num_list = [char for char in value if char.isdigit()]
        return float(''.join(num_list)) if num_list else None
    elif isinstance(value, (int, float)):
        return float(value)
    else:
        return None
if 'age ' in df.columns:
    df['age '] = df['age '].apply(clean_numeric_age)
    invalid_values = df['age '].loc[~df['age '].apply(lambda x: isinstance(x, (int, float)))]
    if not invalid_values.empty:
        print("Warning: There are missing or invalid values in the 'age ' column. Handling them appropriately.")
        df = df.drop(index=invalid_values.index)
    for feature in df.columns[:-1]:
        for Class in df['buys_computer'].unique():
            s = df[df['buys_computer'] == Class][feature]
            s_cleaned = s.dropna()
            if s_cleaned.empty:
                print(f"Class: {Class}, Feature: {feature}")
                print([])
            else:
                mean = s_cleaned.mean()
                std_dev = s_cleaned.std()
                class_cond_density = norm.pdf(s_cleaned, loc=mean, scale=std_dev)
                print(f"Class: {Class}, Feature: {feature}")
                print(class_cond_density)
else:
    print("Error: 'age ' column not found in the DataFrame.")


Class: no, Feature: age 
[0.05578759 0.05578759 0.03997355 0.05578759 0.03997355]
Class: yes, Feature: age 
[0.00013992 0.00017124 0.00017124 0.00013992 0.00017036 0.00017124
 0.00017036 0.00013992 0.00013992]


TypeError: Could not convert highhighlowmediummedium to numeric

In [10]:
#A3
from scipy.stats import chi2_contingency
contingency_table=pd.crosstab(index=df['age '],columns=[df['income'],df['student'],df['credit_rating'],df['buys_computer']])
chi2,p,dof,expected=chi2_contingency(contingency_table)
print("chi square value ", chi2)
print(" p value ", p)
print(" degrees of freedom are ", dof)
print("expected frequencies ", expected)

chi square value  25.199999999999992
 p value  0.3949583040945456
 degrees of freedom are  24
expected frequencies  [[0.35714286 0.35714286 0.35714286 0.35714286 0.35714286 0.35714286
  0.71428571 0.35714286 0.35714286 0.35714286 0.35714286 0.35714286
  0.35714286]
 [0.35714286 0.35714286 0.35714286 0.35714286 0.35714286 0.35714286
  0.71428571 0.35714286 0.35714286 0.35714286 0.35714286 0.35714286
  0.35714286]
 [0.28571429 0.28571429 0.28571429 0.28571429 0.28571429 0.28571429
  0.57142857 0.28571429 0.28571429 0.28571429 0.28571429 0.28571429
  0.28571429]]


In [ ]:
#A4 
from sklearn.model_selection import train_test_split 
from sklearn